# Description

### Context

Patients with Liver disease have been continuously increasing because of excessive consumption of alcohol, inhale of harmful gases, intake of contaminated food, pickles and drugs. This dataset was used to evaluate prediction algorithms in an effort to reduce burden on doctors. 

### Content

This data set contains 416 liver patient records and 167 non liver patient records collected from North East of Andhra Pradesh, India. The "Dataset" column is a class label used to divide groups into liver patient (liver disease) or not (no disease). This data set contains 441 male patient records and 142 female patient records. 
Any patient whose age exceeded 89 is listed as being of age "90".

Columns:

Age of the patient 

Gender of the patient 

Total Bilirubin 

Direct Bilirubin 

Alkaline Phosphotase 

Alamine Aminotransferase 

Aspartate Aminotransferase 

Total Protiens 

Albumin 

Albumin and Globulin Ratio 

Dataset: field used to split the data into two sets (patient with liver disease, or no disease)

### Inspiration

Use these patient records to determine which patients have liver disease and which ones do not.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as st 
import os
from tqdm import tqdm
#os.chdir('D:\Shashi Katteri\Desktop\data sets to work on\indian-liver-patient-records')

In [ ]:
df=pd.read_csv('/kaggle/input/indian-liver-patient-records/indian_liver_patient.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.Dataset.value_counts()

In [ ]:
df1=df
df.columns

In [ ]:
df1['Dataset']=df1['Dataset'].apply(lambda x:0 if x==2 else x)

In [ ]:
df1.Dataset.value_counts()

In [ ]:
df1=df1.fillna(method='bfill')

In [ ]:
df1.info()

In [ ]:
sns.pairplot(data=df1,diag_kind='kde',hue='Dataset')

In [ ]:
for i in df1.columns:
    sns.boxplot(y=df1[i],x=df1['Dataset'])
    plt.show()

In [ ]:
df1=pd.get_dummies(df1,columns=['Gender'],drop_first=True)

In [ ]:
sns.countplot(df1['Gender_Male'],hue=df1['Dataset'])

In [ ]:
df1.corr()

In [ ]:
df1.info()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,roc_curve,roc_auc_score,confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score

In [ ]:
x=df1.drop('Dataset',axis=1)
y=df1['Dataset']

In [ ]:
def mod_score(algo,x,y,params=None):
    cv1=cross_val_score(algo,x,y,cv=5,scoring='accuracy')
    cv2=cross_val_score(algo,x,y,cv=5,scoring='roc_auc')
    print('Accuracy : ',cv1.mean())
    print('ROC AUC score : ',cv2.mean())
    
    

In [ ]:
def rand_search(algo,params,x,y):
    rs=RandomizedSearchCV(algo,param_distributions=params,random_state=0,n_jobs=-1,n_iter=100,scoring='roc_auc',cv=10)
    mod=rs.fit(x,y)
    print(mod.best_score_)
    return mod.best_params_

In [ ]:
rfc_params={'n_estimators':st.randint(50,300),
    'criterion':['gini','entropy'],
    'max_depth':st.randint(2,20),
    'min_samples_split':st.randint(2,100),
    'min_samples_leaf':st.randint(2,100)}
lgb_params={ 'num_leaves':st.randint(31,60),
   'max_depth':st.randint(2,20),
    'learning_rate':st.uniform(0,1),
    'n_estimators':st.randint(50,300),
    'min_split_gain':st.uniform(0,0.3)}

In [ ]:
rbp=rand_search(RandomForestClassifier(),rfc_params,x,y)

In [ ]:
lbp=rand_search(LGBMClassifier(),lgb_params,x,y)

In [ ]:
models={'Logistic Regression':LogisticRegression(solver='liblinear'),'Random Forest':RandomForestClassifier(**rbp),
       'Light GBM(Boosting)':LGBMClassifier(**lbp),'Gausian Naive Bayes':GaussianNB()
       }
params={'Logistic Regression':{'solver':'liblinear'},'Random Forest':rbp,
       'Light GBM(Boosting)':lbp,'Gausian Naive Bayes':None}

In [ ]:
for i in models.keys():
    print(i,'\n')
    mod_score(models[i],x,y)
    print('\n')

In [ ]:
rfc=RandomForestClassifier(**rbp)
mod=rfc.fit(x,y)
y=mod.feature_importances_

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(y=y,x=x.columns)

# Feature engineering

In [ ]:
df3=df1

In [ ]:
#df3=pd.get_dummies(df3,columns=['Gender'],drop_first=True)
#df3=df3.fillna(method='bfill')
df3.info()

In [ ]:
x1=df3.drop('Dataset',axis=1)
y=df3['Dataset']

In [ ]:
#d=x1
ss=StandardScaler()
d=ss.fit_transform(x1)
x1['sum']=d.sum(axis=1)
x1['min']=d.min(axis=1)
x1['max']=d.max(axis=1)
x1['skew']=st.skew(d,axis=1)
x1['kurt']=st.kurtosis(d,axis=1)
x1['std']=d.std(axis=1)

In [ ]:
x1.info()

In [ ]:
rbp1=rand_search(RandomForestClassifier(),rfc_params,x1,y)

In [ ]:
lbp1=rand_search(LGBMClassifier(),lgb_params,x1,y)

In [ ]:
models1={'Logistic Regression':LogisticRegression(solver='liblinear'),'Random Forest':RandomForestClassifier(**rbp1),
       'Light GBM(Boosting)':LGBMClassifier(**lbp1),'Gausian Naive Bayes':GaussianNB()
       }
params={'Logistic Regression':{'solver':'liblinear'},'Random Forest':rbp,
       'Light GBM(Boosting)':lbp,'Gausian Naive Bayes':None}

In [ ]:
for i in models1.keys():
    print(i,'\n')
    mod_score(models1[i],x1,y)
    print('\n')

In [ ]:
lgb=LGBMClassifier(**lbp1)
mod=lgb.fit(x1,y)
f=mod.feature_importances_
plt.figure(figsize=(15,15))
sns.barplot(y=f,x=x1.columns)

The new features created significantly improved the models accuracy. When we give the model more processed and unique information it performs better. In this case we can see that the min column plays an important role. It was also crucial that there were no redundant columns as they may affect the new features created.